In [586]:
import numpy as np
import networkx as nx
from pyvis.network import Network
import math
import community as community_louvain

import seaborn as sns
import pandas as pd
import os

os.chdir(os.path.expanduser('~/Documents/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import xmltodict
import os
import requests
from bs4 import BeautifulSoup
from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult, MetabolismRedux


FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [587]:
# Change the environment name here
env_name = 'Hydrogen_fumarate_-O2_Glc'

In [588]:
condition_df = pd.read_excel("ecoli/experiments/new_environments_test/done.xlsx")
condition_df["Add"] = condition_df["Add"].apply(lambda x: set(str(x).split(", ") if x == x else set()))
condition_df["Remove"] = condition_df["Remove"].apply(lambda x: set(str(x).split(", ") if x == x else set()))

row_num = condition_df[condition_df['Environment'] == env_name].index[0]
important_add = condition_df["Add"].loc[row_num]
important_remove = condition_df["Remove"].loc[row_num]

In [589]:
time = '50'
date = '2023-07-21'
experiment = 'fba_new_environments'
entry = f'{experiment}_{env_name}_{time}_{date}'
folder = f'out/fba_new_env/{entry}/'

f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux']
stoichiometry = metabolism.stoichiometry

output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

sim_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns = metabolism.reaction_names)
sim_fluxes = pd.DataFrame(sim_fluxes.loc[24, :].abs().sort_values(ascending=False))

metabolites_idx = {species: i for i, species in enumerate(metabolism.metabolite_names)}
index_to_met = {v: k for k, v in metabolites_idx.items()}

reaction_idx = {reaction: i for i, reaction in enumerate(metabolism.reaction_names)}
index_to_rxn = {v: k for k, v in reaction_idx.items()}

metabolism.catalyst_ids = metabolism.parameters['catalyst_ids']
catalyst_idx = {catalyst: i for i, catalyst in enumerate(metabolism.catalyst_ids)}

In [590]:
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).loc[24, :]

In [591]:
heatmap_df = pd.read_excel("out/new_env_analysis/heatmap.xlsx")
heatmap_df = heatmap_df[heatmap_df[env_name] != 0]

## Building the graph

In [592]:
G = nx.DiGraph()

met_length = len(metabolism.metabolite_names)
reaction_length = np.shape(stoichiometry)[1]
met_not_include = ["ATP[c]", "WATER[c]", "PROTON[p]", "Pi[c]", "PROTON[c]", "ADP[c]"]
# met_not_include = []
do_not_include = []

for met in met_not_include:
    do_not_include.append(metabolites_idx[met])

def plot_reaction(G, reaction, flux, color):
    index = reaction_idx[reaction]
    reactants = []
    products = []
    clean_name = clean_reaction_name(reaction)
    for k in range(met_length):
        if stoichiometry[k, index] > 0 and index_to_met[k] not in met_not_include:
            products.append(index_to_met[k])
        elif stoichiometry[k, index] < 0 and index_to_met[k] not in met_not_include:
            reactants.append(index_to_met[k])

    for reactant in reactants:
        # nt.add_node(reactant, label=reactant, color='lightblue', shape='star', size=10)
        # nt.add_node(clean_name, label=clean_name, color='lightblue', shape='dot', size=10)
        # nt.add_edge(reactant, clean_name, color=color, title=flux, arrowStrikethrough=False)

        G.add_edge(reactant, clean_name, color=color, title=flux)

    for product in products:
        # nt.add_node(product, label=product, color='lightblue', shape='star', size=10)
        # nt.add_node(clean_name, label=clean_name, color='lightblue', shape='dot', size=10)
        # nt.add_edge(clean_name, product, color=color, title=flux, arrowStrikethrough=False)

        G.add_edge(clean_name, product, color=color, title=flux)

In [593]:
# Not currently used in this analysis but can be useful later

def clean_reaction_name(reaction):
    if '/' in reaction:
        index = reaction.find('RXN')
        if (reaction[index+3] == '-' and reaction[index+4].isdigit()) or (reaction[index+4] == '-' and reaction[index+5].isdigit()):
            index = index + 4
            if reaction[index] == '-':
                index += 1
            while reaction[index].isdigit():
                index += 1
            reaction = reaction[:index]
        else:
            reaction = reaction[:index+3]
    elif '__' in reaction:
        reaction = reaction[:reaction.find('__')]
    elif '(reverse)' in reaction:
        reaction = reaction[:reaction.find('(reverse)') - 1]
    elif '[' in reaction:
        reaction = reaction[:reaction.find('[') - 1]
    elif ' ' in reaction:
        reaction = reaction[:reaction.find('[')]
    return reaction


In [594]:
k = 30
heatmap_df.sort_values(by=[env_name], ascending=False, inplace=True)
top_increase = [heatmap_df.iloc[i, 0] for i in range(k)]
bottom_increase = [heatmap_df.iloc[i, 0] for i in range(-1, -1*k - 1, -1)]

if "Non-zero Values for each environment" in top_increase:
    top_increase.remove("Non-zero Values for each environment")
elif "Non-zero Values for each environment" in bottom_increase:
    bottom_increase.remove("Non-zero Values for each environment")

for reaction in top_increase:
    row_num = heatmap_df[heatmap_df['Reactions'] == reaction].index[0]
    plot_reaction(G, reaction, math.log(heatmap_df[env_name].loc[row_num], 10), 'green')

for reaction in bottom_increase:
    row_num = heatmap_df[heatmap_df['Reactions'] == reaction].index[0]
    plot_reaction(G, reaction, math.log(heatmap_df[env_name].loc[row_num], 10), 'red')

# nx.write_graphml(G, f'out/new_env_analysis/{env_name}_top10_graph.graphml')

# nt = Network('1000px', '2000px', notebook=True, cdn_resources='in_line')
# nt.from_nx(G)

## Graph Analysis using NetworkX

In [595]:
degree = nx.degree_centrality(G)
sort_degree = sorted(degree.items(), key=lambda x:x[1], reverse=True)
print("Degree Centrality: "+ str(sort_degree) + "\n")

closeness = nx.closeness_centrality(G)
sort_closeness = sorted(closeness.items(), key=lambda x:x[1], reverse=True)
print("Closeness Centrality: "+ str(sort_closeness))

print("Average Clustering: " + str(nx.average_clustering(G)))
print("Transitivity: " + str(nx.transitivity(G)))

Degree Centrality: [('R15-RXN', 0.06622516556291391), ('PYRUVATE[c]', 0.046357615894039736), ('PYFLAVOXRE-RXN', 0.039735099337748346), ('CARBON-DIOXIDE[c]', 0.039735099337748346), ('ACETYL-COA[c]', 0.033112582781456956), ('NADH[c]', 0.033112582781456956), ('NAD[c]', 0.033112582781456956), ('MALIC-NADP-RXN', 0.033112582781456956), ('GAP[c]', 0.033112582781456956), ('ACETALD-DEHYDROG-RXN', 0.033112582781456956), ('FUM[c]', 0.026490066225165563), ('R601-RXN', 0.026490066225165563), ('TRANS-RXN-106', 0.026490066225165563), ('FORMATE[c]', 0.026490066225165563), ('CO-A[c]', 0.026490066225165563), ('PYRUVFORMLY-RXN', 0.026490066225165563), ('NADPH[c]', 0.026490066225165563), ('FLAVONADPREDUCT-RXN', 0.026490066225165563), ('NADP[c]', 0.026490066225165563), ('MALATE-DEH-RXN', 0.026490066225165563), ('GAPOXNPHOSPHN-RXN', 0.026490066225165563), ('DIHYDROOROTATE-DEHYDROGENASE-RXN', 0.026490066225165563), ('THIOREDOXIN-REDUCT-NADPH-RXN', 0.026490066225165563), ('RXN0-5055', 0.026490066225165563), (

In [596]:
sort_closeness = sort_closeness[:8]
nt = Network('1000px', '1500px', notebook=True, cdn_resources='in_line', directed=True)
nt.from_nx(G)

for tup in sort_closeness:
    nt.get_node(tup[0])['color'] = 'blue'

for n in nt.nodes:
    if n['label'] in metabolites_idx:
        n['shape'] = 'star'
    if n['label'] in important_add:
        n['color'] = 'orange'
    if n['label'] in important_remove:
        n['color'] = 'purple'


nt.toggle_physics(True)
nt.show_buttons(filter_=['physics'])
nt.show(f'out/new_env_analysis/flux_visualization/{env_name}_top{k}_network.html')

# plt.figure(figsize=(25, 25))
# nx.draw_spring(G, with_labels=True, font_size=10, node_size=1000, node_color='lightblue', edge_color='black', width=1.5)
# plt.savefig(f'out/new_env_analysis/{env_name}_top10_graph.png', dpi=300)
# plt.show()

out/new_env_analysis/flux_visualization/Hydrogen_fumarate_-O2_Glc_top30_network.html
